# Analyzing Image Metadata

Exchangeable image file format (Exif) has been a standard around since 1998 to include metadata in image file formats like JPEG, WAV, HEIC, and WEBP.  Digital cameras and smart phones with GPS receivers have also included geolocation coordinates.  This is a good application of reverse geocoding.

![image.jpg](./image.jpg)

Read this blog post for more background:

https://developer.here.com/blog/getting-started-with-geocoding-exif-image-metadata-in-python3

In [1]:
from PIL import Image

def get_exif(filename):
    image = Image.open(filename)
    image.verify()
    return image._getexif()

In [5]:
exif = get_exif('image.jpg')
exif

{34853: {1: 'N',
  2: ((36, 1), (29, 1), (902, 100)),
  3: 'W',
  4: ((114, 1), (31, 1), (2481, 100)),
  5: b'\x00',
  6: (1030399, 1770),
  7: ((0, 1), (35, 1), (40, 1)),
  12: 'K',
  13: (0, 1),
  16: 'T',
  17: (347654, 981),
  23: 'T',
  24: (347654, 981),
  29: '2019:01:22',
  31: (5, 1)},
 296: 2,
 34665: 206,
 271: 'Apple',
 272: 'iPhone 8',
 305: '12.1.2',
 274: 1,
 306: '2019:01:21 16:35:40',
 531: 1,
 282: (72, 1),
 283: (72, 1),
 36864: b'0221',
 37121: b'\x01\x02\x03\x00',
 37377: (247576, 35831),
 36867: '2019:01:21 16:35:40',
 36868: '2019:01:21 16:35:40',
 37378: (54823, 32325),
 37379: (53389, 8769),
 37380: (0, 1),
 37383: 5,
 37385: 16,
 37386: (399, 100),
 40961: 65535,
 40962: 4032,
 41989: 28,
 41990: 0,
 40963: 3024,
 37521: '561',
 37522: '561',
 37396: (2015, 1511, 2217, 1330),
 41495: 2,
 33434: (1, 120),
 33437: (9, 5),
 41729: b'\x01',
 34850: 2,
 34855: 25,
 41986: 0,
 40960: b'0100',
 41987: 0,
 42034: ((4183519, 1048501), (4183519, 1048501), (9, 5), (9, 5)

In [4]:
from PIL.ExifTags import TAGS

def get_labeled_exif(exif):
    labeled = {}
    for (key, val) in exif.items():
        labeled[TAGS.get(key)] = val
        
    return labeled

In [6]:
exif = get_exif('image.jpg')
labeled = get_labeled_exif(exif)
labeled

{'GPSInfo': {1: 'N',
  2: ((36, 1), (29, 1), (902, 100)),
  3: 'W',
  4: ((114, 1), (31, 1), (2481, 100)),
  5: b'\x00',
  6: (1030399, 1770),
  7: ((0, 1), (35, 1), (40, 1)),
  12: 'K',
  13: (0, 1),
  16: 'T',
  17: (347654, 981),
  23: 'T',
  24: (347654, 981),
  29: '2019:01:22',
  31: (5, 1)},
 'ResolutionUnit': 2,
 'ExifOffset': 206,
 'Make': 'Apple',
 'Model': 'iPhone 8',
 'Software': '12.1.2',
 'Orientation': 1,
 'DateTime': '2019:01:21 16:35:40',
 'YCbCrPositioning': 1,
 'XResolution': (72, 1),
 'YResolution': (72, 1),
 'ExifVersion': b'0221',
 'ComponentsConfiguration': b'\x01\x02\x03\x00',
 'ShutterSpeedValue': (247576, 35831),
 'DateTimeOriginal': '2019:01:21 16:35:40',
 'DateTimeDigitized': '2019:01:21 16:35:40',
 'ApertureValue': (54823, 32325),
 'BrightnessValue': (53389, 8769),
 'ExposureBiasValue': (0, 1),
 'MeteringMode': 5,
 'Flash': 16,
 'FocalLength': (399, 100),
 'ColorSpace': 65535,
 'ExifImageWidth': 4032,
 'FocalLengthIn35mmFilm': 28,
 'SceneCaptureType': 0,
 '

In [8]:
from PIL.ExifTags import GPSTAGS

def get_geotagging(exif):
    if not exif:
        raise ValueError("No EXIF metadata found")
        
    geotagging = {}
    for (idx, tag) in TAGS.items():
        if tag == 'GPSInfo':
            if idx not in exif:
                raise ValueError("No EXIF geotagging found")

            for (key, val) in GPSTAGS.items():
                if key in exif[idx]:
                    geotagging[val] = exif[idx][key]

    return geotagging

In [9]:
exif = get_exif('image.jpg')
geotags = get_geotagging(exif)
geotags

{'GPSLatitudeRef': 'N',
 'GPSLatitude': ((36, 1), (29, 1), (902, 100)),
 'GPSLongitudeRef': 'W',
 'GPSLongitude': ((114, 1), (31, 1), (2481, 100)),
 'GPSAltitudeRef': b'\x00',
 'GPSAltitude': (1030399, 1770),
 'GPSTimeStamp': ((0, 1), (35, 1), (40, 1)),
 'GPSSpeedRef': 'K',
 'GPSSpeed': (0, 1),
 'GPSImgDirectionRef': 'T',
 'GPSImgDirection': (347654, 981),
 'GPSDestBearingRef': 'T',
 'GPSDestBearing': (347654, 981),
 'GPSDateStamp': '2019:01:22',
 'GPSHPositioningError': (5, 1)}

In [10]:
def get_decimal_from_dms(dms, ref):

    degrees = dms[0][0] / dms[0][1]
    minutes = dms[1][0] / dms[1][1] / 60.0
    seconds = dms[2][0] / dms[2][1] / 3600.0

    if ref in ['S', 'W']:
        degrees = -degrees
        minutes = -minutes
        seconds = -seconds

    return round(degrees + minutes + seconds, 5)

def get_coordinates(geotags):
    lat = get_decimal_from_dms(geotags['GPSLatitude'], geotags['GPSLatitudeRef'])
    lon = get_decimal_from_dms(geotags['GPSLongitude'], geotags['GPSLongitudeRef'])

    return (lat,lon)

In [11]:
exif = get_exif('image.jpg')
geotags = get_geotagging(exif)
coords = get_coordinates(geotags)
coords

(36.48584, -114.52356)

# Try It

Find an image that has some geotags and try to find the corresponding street address with the reverse geocoder examples.